In [150]:
import pandas as pd
import numpy as np

from datetime import datetime

# Import Data

In [151]:
data = pd.read_json("motif_dataset.json", lines=True)

In [152]:
data.columns

Index(['md5', 'sha1', 'sha256', 'reported_hash', 'reported_family', 'aliases',
       'label', 'report_source', 'report_date', 'report_url', 'report_ioc_url',
       'appeared', 'histogram', 'byteentropy', 'strings', 'general', 'header',
       'section', 'imports', 'exports', 'datadirectories'],
      dtype='object')

In [153]:
data.drop(["report_url", "report_ioc_url", "histogram", "label", "report_date", "byteentropy"], axis=1, inplace=True)

In [154]:
data.columns

Index(['md5', 'sha1', 'sha256', 'reported_hash', 'reported_family', 'aliases',
       'report_source', 'appeared', 'strings', 'general', 'header', 'section',
       'imports', 'exports', 'datadirectories'],
      dtype='object')

# Feature Engineering

In [155]:
data.iloc[:, 5:].head(3)

,aliases,report_source,appeared,strings,general,header,section,imports,exports,datadirectories
0,"[shamoon, disttrack]",Symantec,2018-12,"{'numstrings': 8626, 'avlength': 6.32216554602...","{'size': 1921024, 'vsize': 1921024, 'has_debug...","{'coff': {'timestamp': 1322495459, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'NETAPI32.dll': ['NetScheduleJobAdd', 'NetUse...",[],"[{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_..."
1,"[downdelph, delphacy, delphocy]",ESET,2016-10,"{'numstrings': 472, 'avlength': 9.921610169491...","{'size': 111616, 'vsize': 155648, 'has_debug':...","{'coff': {'timestamp': 1399967918, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'oleaut32.dll': ['SysFreeString', 'SysReAlloc...",[],"[{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_..."
2,"[discordiaminer, discordia]",Kaspersky,2017-05,"{'numstrings': 1058, 'avlength': 14.2249527410...","{'size': 245248, 'vsize': 266240, 'has_debug':...","{'coff': {'timestamp': 1496077247, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'KERNEL32.dll': ['Sleep', 'GetLastError', 'Mu...",[],"[{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_..."


In [156]:
data.loc[0, "general"]

{'size': 1921024,
 'vsize': 1921024,
 'has_debug': 0,
 'exports': 0,
 'imports': 163,
 'has_relocations': 1,
 'has_resources': 1,
 'has_signature': 0,
 'has_tls': 0,
 'symbols': 0}

In [157]:
data["file_size_bytes"]=data["general"].agg(lambda x: x["size"])
data["virtual_file_size_bytes"]=data["general"].agg(lambda x: x["vsize"])
data["has_debug"]=data["general"].agg(lambda x: x["has_debug"])
data["num_exports"]=data["general"].agg(lambda x: x["exports"])
data["num_imports"]=data["general"].agg(lambda x: x["imports"])
data["has_resources"]=data["general"].agg(lambda x: x["has_resources"])

In [158]:
data.loc[0, "strings"]

{'numstrings': 8626,
 'avlength': 6.322165546023649,
 'printabledist': [1545,
  409,
  383,
  366,
  465,
  331,
  422,
  552,
  415,
  410,
  351,
  376,
  520,
  488,
  446,
  373,
  634,
  537,
  385,
  457,
  1315,
  456,
  417,
  536,
  481,
  414,
  470,
  482,
  468,
  442,
  488,
  508,
  647,
  530,
  746,
  542,
  557,
  513,
  506,
  791,
  536,
  452,
  557,
  421,
  538,
  481,
  548,
  375,
  592,
  394,
  476,
  624,
  469,
  374,
  473,
  502,
  484,
  438,
  433,
  439,
  360,
  381,
  420,
  430,
  455,
  802,
  469,
  1074,
  543,
  1155,
  488,
  522,
  1362,
  1219,
  472,
  569,
  818,
  533,
  696,
  769,
  561,
  396,
  855,
  720,
  975,
  546,
  427,
  750,
  1226,
  494,
  389,
  1194,
  429,
  485,
  361,
  410],
 'printables': 54535,
 'entropy': 6.475968360900879,
 'paths': 0,
 'urls': 0,
 'registry': 0,
 'MZ': 18}

In [159]:
data["num_strings"]=data["strings"].agg(lambda x: x["numstrings"])
data["avg_string_len"]=data["strings"].agg(lambda x: x["avlength"])
data["num_printable_strings"]=data["strings"].agg(lambda x: x["printables"])
data["num_path_strings"]=data["strings"].agg(lambda x: x["paths"])
data["num_url_strings"]=data["strings"].agg(lambda x: x["urls"])
data["num_registry_strings"]=data["strings"].agg(lambda x: x["registry"])

In [160]:
# data.head(4)

In [161]:
data.loc[789, "section"]

{'entry': '.text',
 'sections': [{'name': '.text',
   'size': 40960,
   'entropy': 5.680471600569829,
   'vsize': 38364,
   'props': ['CNT_CODE', 'MEM_EXECUTE', 'MEM_READ']},
  {'name': '.data',
   'size': 4096,
   'entropy': -0.0,
   'vsize': 6544,
   'props': ['CNT_INITIALIZED_DATA', 'MEM_READ', 'MEM_WRITE']},
  {'name': '.rsrc',
   'size': 237568,
   'entropy': 5.405046225418915,
   'vsize': 235498,
   'props': ['CNT_INITIALIZED_DATA', 'MEM_READ']}]}

In [162]:
data["entry_point"]=data["section"].agg(lambda x: x["entry"])

In [163]:
sec = set()
x = data[["section"]].copy()
for row in x["section"]:
    for i in row["sections"]:
        sec.add(i["name"])
        
        
sec

{'',
 '\x03}AI?s\x0f',
 '\x047K"PY\x16\t',
 '\x15J\x03\x14\x17\x0e\x17W',
 '\x19\x19\x0cLDP\x04@',
 ' ',
 '   ',
 '        ',
 '"a%\x01\x0e1\x10w',
 '$LI[EU^-',
 '$hVzesTS',
 "'5?#(IbB",
 "'HU\x1f,\x06a\x1b",
 '(w:$>1]l',
 '*^I)JH`\\',
 '+XT|sS4',
 '-4',
 '.',
 '.00cfg',
 '.1LA',
 '.8010',
 '.8011',
 '.BSS',
 '.BSS3',
 '.BSS4',
 '.CRT',
 '.HOKKES',
 '.HOOKDAT',
 '.MPRESS1',
 '.MPRESS2',
 '.NewIT',
 '.SCY',
 '.SE',
 '.Silvana',
 '.UPX',
 '.UPX0',
 '.UPX1',
 '.UPX2',
 '.X1D',
 '.Xyl2k!',
 '.a050',
 '.a051',
 '.adata',
 '.api',
 '.arch',
 '.aspack',
 '.barusec',
 '.bin',
 '.bss',
 '.bss0',
 '.bxoer',
 '.cctype',
 '.cdata',
 '.cfg',
 '.code',
 '.cotx',
 '.crt',
 '.crt0',
 '.crt1',
 '.cwtnsn',
 '.data',
 '.data1',
 '.data2',
 '.data4',
 '.dec',
 '.didat',
 '.didata',
 '.dll0',
 '.dll1',
 '.dtext',
 '.edata',
 '.eh_fram',
 '.enigma1',
 '.enigma2',
 '.erloc',
 '.eyolc',
 '.f',
 '.fdata',
 '.fg15',
 '.flat',
 '.gcode',
 '.gdata',
 '.gdb0',
 '.gdb1',
 '.gerc',
 '.gfids',
 '.ggeE',
 '.gig',
 '.g

In [164]:
def GetSectionData(x, want): # (section, key)
    sections = x["sections"]
    # print(x)
    for section in sections:
        if want[0] != section["name"]:
            return -1
        else:
            return section[want[1]]
    

In [165]:
data[".text_size"]=data["section"].agg(GetSectionData, 0, (".text", "size"))
data[".rdata_size"]=data["section"].agg(GetSectionData, 0, (".rdata", "size"))
data[".data_size"]=data["section"].agg(GetSectionData, 0, (".data", "size"))
data[".rsrc_size"]=data["section"].agg(GetSectionData, 0, (".rsrc", "size"))
data[".reloc_size"]=data["section"].agg(GetSectionData, 0, (".reloc", "size"))

In [166]:
data[".text_vsize"]=data["section"].agg(GetSectionData, 0, (".text", "vsize"))
data[".rdata_vsize"]=data["section"].agg(GetSectionData, 0, (".rdata", "vsize"))
data[".data_vsize"]=data["section"].agg(GetSectionData, 0, (".data", "vsize"))
data[".rsrc_vsize"]=data["section"].agg(GetSectionData, 0, (".rsrc", "vsize"))
data[".reloc_vsize"]=data["section"].agg(GetSectionData, 0, (".reloc", "vsize"))

In [167]:
data[".text_entropy"]=data["section"].agg(GetSectionData, 0, (".text", "entropy"))
data[".rdata_entropy"]=data["section"].agg(GetSectionData, 0, (".rdata", "entropy"))
data[".data_entropy"]=data["section"].agg(GetSectionData, 0, (".data", "entropy"))
data[".rsrc_entropy"]=data["section"].agg(GetSectionData, 0, (".rsrc", "entropy"))
data[".reloc_entropy"]=data["section"].agg(GetSectionData, 0, (".reloc", "entropy"))

In [168]:
data.columns

Index(['md5', 'sha1', 'sha256', 'reported_hash', 'reported_family', 'aliases',
       'report_source', 'appeared', 'strings', 'general', 'header', 'section',
       'imports', 'exports', 'datadirectories', 'file_size_bytes',
       'virtual_file_size_bytes', 'has_debug', 'num_exports', 'num_imports',
       'has_resources', 'num_strings', 'avg_string_len',
       'num_printable_strings', 'num_path_strings', 'num_url_strings',
       'num_registry_strings', 'entry_point', '.text_size', '.rdata_size',
       '.data_size', '.rsrc_size', '.reloc_size', '.text_vsize',
       '.rdata_vsize', '.data_vsize', '.rsrc_vsize', '.reloc_vsize',
       '.text_entropy', '.rdata_entropy', '.data_entropy', '.rsrc_entropy',
       '.reloc_entropy'],
      dtype='object')

In [169]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   md5                      3095 non-null   object 
 1   sha1                     3095 non-null   object 
 2   sha256                   3095 non-null   object 
 3   reported_hash            3095 non-null   object 
 4   reported_family          3095 non-null   object 
 5   aliases                  3095 non-null   object 
 6   report_source            3095 non-null   object 
 7   appeared                 3095 non-null   object 
 8   strings                  3095 non-null   object 
 9   general                  3095 non-null   object 
 10  header                   3095 non-null   object 
 11  section                  3095 non-null   object 
 12  imports                  3095 non-null   object 
 13  exports                  3095 non-null   object 
 14  datadirectories         

In [170]:
data.drop(["section", "general", "strings"], axis=1, inplace=True)

In [171]:
data["appeared"]

0       2018-12
1       2016-10
2       2017-05
3       2017-07
4       2018-10
         ...   
3090    2016-08
3091    2017-04
3092    2019-03
3093    2021-01
3094    2018-01
Name: appeared, Length: 3095, dtype: object

In [172]:
data["appeared_2"] = pd.to_datetime(data["appeared"])

In [173]:
# data["appeared"]

In [174]:
# data.info()

In [175]:
data.to_csv("motif_cleaned.csv")

# Import Data

In [177]:
import_data = data[["imports", "appeared_2"]].copy()

In [178]:
import_data.head(5)

,imports,appeared_2
0,"{'NETAPI32.dll': ['NetScheduleJobAdd', 'NetUse...",2018-12-01
1,"{'oleaut32.dll': ['SysFreeString', 'SysReAlloc...",2016-10-01
2,"{'KERNEL32.dll': ['Sleep', 'GetLastError', 'Mu...",2017-05-01
3,"{'VERSION.dll': ['GetFileVersionInfoW', 'VerQu...",2017-07-01
4,"{'KERNEL32.dll': ['GetOEMCP', 'SetErrorMode', ...",2018-10-01


In [ ]:
data_formatted = {}